本代码主要实现利用GEE下载无云影像数据，根据矢量提取对应位置的value等内容

In [1]:
# Import libraries
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

In [2]:
import geopandas as gpd
import pandas as pd 
import numpy as np 

In [3]:
geemap.set_proxy(port = 1081)

In [ ]:
# geemap.update_package()

In [4]:
ee.Authenticate()


Successfully saved authorization token.


In [6]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [7]:
style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

In [15]:
admin1_widget = widgets.Text(
    description='State:',
    value='Tennessee',
    width=200,
    style=style
)

admin2_widget = widgets.Text(
    description='County:',
    value='Knox',
    width=300,
    style=style
)
aoi_widget = widgets.Checkbox(
    value=False,
    description='Use user-drawn AOI',
    style=style
)

download_widget = widgets.Checkbox(
    value=False,
    description='Download chart data',
    style=style
)
def aoi_change(change):
    Map.layers = Map.layers[:4]   
    Map.user_roi = None
    Map.user_rois = None
    Map.draw_count = 0
    admin1_widget.value = ''
    admin2_widget.value = ''
    output_widget.clear_output()
    
aoi_widget.observe(aoi_change, names='value')
band_combo = widgets.Dropdown(
    description='Band combo:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='NIR/Red/Green',
    style=style
)

year_widget = widgets.IntSlider(min=1984, max=2020, value=2010, description='Selected year:', width=400, style=style)

fmask_widget = widgets.Checkbox(
    value=True,
    description='Apply fmask(remove cloud, shadow, snow)',
    style=style
)
nd_options = ['Vegetation Index (NDVI)', 
              'Water Index (NDWI)',
              'Modified Water Index (MNDWI)',
              'Snow Index (NDSI)',
              'Soil Index (NDSI)',
              'Burn Ratio (NBR)',
              'Customized']
nd_indices = widgets.Dropdown(options=nd_options, value='Modified Water Index (MNDWI)', description='Normalized Difference Indes:', style=style)

In [19]:
fmask_widget = widgets.Checkbox(
    value=True,
    description='Apply fmask(remove cloud, shadow, snow)',
    style=style
)


# Normalized Satellite Indices: https://www.usna.edu/Users/oceano/pguth/md_help/html/norm_sat.htm

nd_options = ['Vegetation Index (NDVI)', 
              'Water Index (NDWI)',
              'Modified Water Index (MNDWI)',
              'Snow Index (NDSI)',
              'Soil Index (NDSI)',
              'Burn Ratio (NBR)',
              'Customized']
nd_indices = widgets.Dropdown(options=nd_options, value='Modified Water Index (MNDWI)', description='Normalized Difference Indes:', style=style)

first_band = widgets.Dropdown(
    description='1st band:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='Green',
    style=style
)
second_band = widgets.Dropdown(
    description='2nd band:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='SWIR1',
    style=style
)

nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style
)

nd_color = widgets.ColorPicker(
    concise=False,
    description='Color:',
    value='blue',
    style=style
)
def nd_index_change(change):
    if nd_indices.value == 'Vegetation Index (NDVI)':
        first_band.value = 'NIR'
        second_band.value = 'Red'
    elif nd_indices.value == 'Water Index (NDWI)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'Modified Water Index (MNDWI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'Snow Index (NDSI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'
    elif nd_indices.value == 'Soil Index (NDSI)':
        first_band.value = 'SWIR1'
        second_band.value = 'NIR'        
    elif nd_indices.value == 'Burn Ratio (NBR)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR2'
    elif nd_indices.value == 'Customized':
        first_band.value = None
        second_band.value = None
        
nd_indices.observe(nd_index_change, names='value')

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)
full_widget = widgets.VBox([
    widgets.HBox([admin1_widget, admin2_widget, aoi_widget, download_widget]),
    widgets.HBox([band_combo, year_widget, fmask_widget]),
    widgets.HBox([nd_indices, first_band, second_band, nd_threshold, nd_color]),
    submit
])

In [20]:
full_widget

In [21]:
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click' and not aoi_widget.value:
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = fc.filterBounds(xy)
        
        with output_widget:
            output_widget.clear_output()
        
            try:
                feature = selected_fc.first()
                admin2_id = feature.get('NAME').getInfo()
                statefp = feature.get('STATEFP')
                admin1_fc = ee.Feature(states.filter(ee.Filter.eq('STATEFP', statefp)).first())                
                admin1_id = admin1_fc.get('NAME').getInfo()
                admin1_widget.value = admin1_id
                admin2_widget.value = admin2_id
                Map.layers = Map.layers[:4]        
                geom = selected_fc.geometry()
                layer_name = admin1_id + '-' + admin2_id
                Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)  
                print(layer_name)
            except:
                print('No feature could be found')
                Map.layers = Map.layers[:4]        
            
        Map.default_style = {'cursor': 'pointer'}
    else:
        Map.draw_count = 0

Map.on_interaction(handle_interaction)

In [23]:
def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            admin1_id = admin1_widget.value
            admin2_id = admin2_widget.value
            band1 = first_band.value
            band2 = second_band.value
            selected_year = year_widget.value
            threshold = nd_threshold.value
            bands = band_combo.value.split('/')
            apply_fmask = fmask_widget.value
            palette = nd_color.value
            use_aoi = aoi_widget.value
            download = download_widget.value
            
            if use_aoi:
                if Map.user_roi is not None:
                    roi = Map.user_roi
                    layer_name = 'User drawn AOI'
                    geom = roi
                else:
                    output_widget.clear_output() 
                    print('No user AOI could be found.')
                    return
            else:
            
                statefp = ee.Feature(states.filter(ee.Filter.eq('NAME', admin1_id)).first()).get('STATEFP')
                roi = fc.filter(ee.Filter.And(ee.Filter.eq('NAME', admin2_id), ee.Filter.eq('STATEFP', statefp)))
                layer_name = admin1_id + '-' + admin2_id
                geom = roi.geometry()


            Map.layers = Map.layers[:4]        
            Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)  
            
            images = geemap.landsat_timeseries(roi=roi, start_year=1984, end_year=2020, start_date='01-01', end_date='12-31', apply_fmask=apply_fmask)
            nd_images = images.map(lambda img: img.normalizedDifference([band1, band2]))
            result_images = nd_images.map(lambda img: img.gt(threshold))

            selected_image = ee.Image(images.toList(images.size()).get(selected_year - 1984))
            selected_result_image = ee.Image(result_images.toList(result_images.size()).get(selected_year - 1984)).selfMask()
            
            vis_params = {
                'bands': bands,
                'min': 0,
                'max': 3000
            }
            
            Map.addLayer(selected_image, vis_params, 'Landsat ' + str(selected_year))
            Map.addLayer(selected_result_image, {'palette': palette}, 'Result ' + str(selected_year))

            
            def cal_area(img):
                pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e4)
                img_area = pixel_area.reduceRegion(**{
                    'geometry': geom,
                    'reducer': ee.Reducer.sum(),
                    'scale': 1000,
                    'maxPixels': 1e12,
                    'bestEffort': True
                })
                return img.set({'area': img_area})
            
            areas = result_images.map(cal_area)
            stats = areas.aggregate_array('area').getInfo()
            x = list(range(1984, 2021))
            y = [item.get('nd') for item in stats]
            
            fig = plt.figure(1)
            fig.layout.height = '270px'
            plt.clear()
            plt.plot(x, y)
            plt.title('Temporal trend (1984-2020)')
            plt.xlabel('Year')
            plt.ylabel('Area (ha)')
            
            output_widget.clear_output()            

            plt.show()
            
            if download:
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_name = 'chart_' + geemap.random_string() + '.csv'
                out_csv = os.path.join(out_dir, out_name)
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)
                with open(out_csv, 'w') as f:
                    f.write('year, area (ha)\n')
                    for index, item in enumerate(x):
                        line = '{},{:.2f}\n'.format(item, y[index])
                        f.write(line) 
                link = geemap.create_download_link(
                    out_csv, title="Click here to download the chart data: ")
                display(link)
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')        

        Map.default_style = {'cursor': 'default'}

submit.on_click(submit_clicked)

In [25]:
submit

Button(button_style='primary', description='Submit', style=ButtonStyle(), tooltip='Click me')

In [6]:
#roi define roi
roi = ee.FeatureCollection('users/xtnncherish/workstation/hhy')
Map.addLayer(roi, {}, 'HHY')

NameError: name 'ee' is not defined

In [8]:
# ee.Algorithms.Landsat.simpleComposite
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
                .filterDate('2020-06-01', '2020-10-15')\
                .filterBounds(roi)\
                .filter(ee.Filter.lt('CLOUD_COVER',20))

In [10]:
composite_img = ee.Algorithms.Landsat.simpleComposite(collection).clip(roi)

In [12]:
# visiulation
Map.addLayer(composite_img, {'bands': ['B4', 'B3', 'B2'], max: 128}, 'hhy_img')

In [17]:
img_file = 'F:\wb\lwm\hhy_img.tif'
geemap.ee_export_image(composite_img, filename = img_file, scale = 30, region = roi.geometry(), file_per_band = False)

Generating URL ...
An error occurred while downloading.
Pixel grid dimensions (28133x16291) must be less than or equal to 10000.


In [8]:
points = ee.FeatureCollection('users/xtnncherish/nj_points')
Map.addLayer(points, {}, "points", False)

In [24]:
lulc = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate('2018-01-01','2019-01-01').select('LC_Type1').mosaic()
dem = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
ndvi = ee.ImageCollection("MODIS/006/MOD13Q1").filterDate('2019-06-01','2019-11-01').select('NDVI').median()
lst = ee.ImageCollection("JAXA/GCOM-C/L3/LAND/LST/V1") .filterDate('2019-06-01','2019-11-01').select('LST_AVE').median()

In [25]:
Map.addLayer(lst,{},'LST')

In [26]:
slope = ee.Terrain.slope(dem)

In [27]:
data_list = [lulc, slope, ndvi, lst]
data_name = ['lulc', 'slope', 'ndvi', 'lst']
data_scale = [500, 30, 100, 500]
for i in range(4):
    work_dir = os.path.expanduser('~/Downloads')
    out_shp = os.path.join(work_dir,'points_{}.shp'.format(data_name[i]))
    geemap.extract_values_to_points(in_fc = points, image = data_list[i], scale =data_scale[i], out_fc = out_shp)

# work_dir = os.path.expanduser('~/Downloads')
# out_shp = os.path.join(work_dir, 'points_lulc.shp')
# geemap.extract_values_to_points(in_fc=points, image=lulc, out_fc = out_shp)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_lulc.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_slope.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_ndvi.shp
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_lst.shp


In [11]:
work_dir = os.path.expanduser('~/Downloads')
out_shp = os.path.join(work_dir,'points_ndvi.shp')
geemap.extract_values_to_points(in_fc = points, image = ndvi, scale = 100, out_fc = out_shp)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Admin\Downloads\points_ndvi.shp


In [28]:
points_lulc = gpd.read_file(r'C:\Users\Admin\Downloads\points_lulc.shp')
points_slope = gpd.read_file(r'C:\Users\Admin\Downloads\points_slope.shp')
points_ndvi = gpd.read_file(r'C:\Users\Admin\Downloads\points_ndvi.shp')
points_lst = gpd.read_file(r'C:\Users\Admin\Downloads\points_lst.shp')

In [33]:
print(points_lst.head())
# print(points_slope.head())
# print(points_ndvi.head())

       OBJECTID      COR_Y  LST_AVE      COR_X                   geometry
6850          1  41.340586  14732.0  79.960606  POINT (79.96129 41.34272)
13952         2  41.340799  14732.0  79.984496  POINT (79.98375 41.34272)
14684         3  41.377420  14629.0  80.055649  POINT (80.05561 41.37865)
392           4  41.377614  14629.0  80.079553  POINT (80.07807 41.37865)
12501         5  41.377803  14646.0  80.103458  POINT (80.10502 41.37865)


In [30]:
points_lulc.sort_values('OBJECTID', inplace = True)
points_slope.sort_values('OBJECTID', inplace = True)
points_ndvi.sort_values('OBJECTID', inplace = True)

In [31]:
points_lst.sort_values('OBJECTID', inplace = True)

In [37]:
nj_points = gpd.read_file(r'F:\python_code\gdal_gee\GEE_LEARN\nj_points.shp')
nj_points.head()

,OBJECTID,COR_Y,COR_X,geometry
0,1,41.340586,79.960606,POINT (79.96061 41.34059)
1,2,41.340799,79.984496,POINT (79.98450 41.34080)
2,3,41.377420,80.055649,POINT (80.05565 41.37742)
3,4,41.377614,80.079553,POINT (80.07955 41.37761)
4,5,41.377803,80.103458,POINT (80.10346 41.37780)


In [38]:
nj_points['LULC'] = points_lulc['LC_Type1'].values
nj_points['SLOPE'] = points_slope['slope'].values
nj_points['NDVI'] = points_ndvi['NDVI'].values
nj_points['LST'] = points_lst['LST_AVE'].values

In [39]:
nj_points.head()

,OBJECTID,COR_Y,COR_X,geometry,LULC,SLOPE,NDVI,LST
0,1,41.340586,79.960606,POINT (79.96061 41.34059),10,1.544394,3178.0,14732.0
1,2,41.340799,79.984496,POINT (79.98450 41.34080),15,3.086556,699.0,14732.0
2,3,41.377420,80.055649,POINT (80.05565 41.37742),10,2.780288,2347.0,14629.0
3,4,41.377614,80.079553,POINT (80.07955 41.37761),10,0.927410,5296.0,14629.0
4,5,41.377803,80.103458,POINT (80.10346 41.37780),10,1.544959,2647.0,14646.0


In [21]:
nj_points.crs

<Geographic 2D CRS: EPSG:4610>
Name: Xian 1980
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: China - onshore
- bounds: (73.62, 18.11, 134.77, 53.56)
Datum: Xian 1980
- Ellipsoid: IAG 1975
- Prime Meridian: Greenwich

In [22]:
import os
os.getcwd()

'f:\\python_code\\gdal_gee\\GEE_LEARN'

In [40]:

nj_points.to_file(driver = 'ESRI Shapefile', filename = 'nj_points_value0.shp')

# gdf.to_file(driver = 'ESRI Shapefile', filename = input_path)